In [1]:
%matplotlib inline

In [2]:
import nltk

In [3]:
text = '''
he PRP B-NP
accepted VBD B-VP
the DT B-NP
position NN I-NP
of IN B-PP
vice NN B-NP
chairman NN I-NP
of IN B-PP
Carlyle NNP B-NP
Group NNP I-NP
, , O
a DT B-NP
merchant NN I-NP
banking NN I-NP
concern NN I-NP
. . O
'''

In [4]:
nltk.chunk.conllstr2tree(text, chunk_types=['NP']).draw()

In [5]:
from nltk.corpus import conll2000

In [6]:
print(conll2000.chunked_sents('train.txt')[99])
# (S
#   (PP Over/IN)
#   (NP a/DT cup/NN)
#   (PP of/IN)
#   (NP coffee/NN)
#   ,/,
#   (NP Mr./NNP Stone/NNP)
#   (VP told/VBD)
#   (NP his/PRP$ story/NN)
#   ./.)

(S
  (PP Over/IN)
  (NP a/DT cup/NN)
  (PP of/IN)
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  (VP told/VBD)
  (NP his/PRP$ story/NN)
  ./.)


In [7]:
print(conll2000.chunked_sents('train.txt', chunk_types=['NP'])[99])
# (S
#   Over/IN
#   (NP a/DT cup/NN)
#   of/IN
#   (NP coffee/NN)
#   ,/,
#   (NP Mr./NNP Stone/NNP)
#   told/VBD
#   (NP his/PRP$ story/NN)
#   ./.)

(S
  Over/IN
  (NP a/DT cup/NN)
  of/IN
  (NP coffee/NN)
  ,/,
  (NP Mr./NNP Stone/NNP)
  told/VBD
  (NP his/PRP$ story/NN)
  ./.)


In [8]:
from nltk.corpus import conll2000

In [9]:
cp = nltk.RegexpParser("")

In [10]:
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])

In [11]:
print(cp.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  43.4%
#     Precision:      0.0%
#     Recall:         0.0%
#     F-Measure:      0.0%

ChunkParse score:
    IOB Accuracy:  43.4%%
    Precision:      0.0%%
    Recall:         0.0%%
    F-Measure:      0.0%%


In [12]:
grammar = r"NP: {<[CDJNP].*>+}"

In [13]:
cp = nltk.RegexpParser(grammar)

In [14]:
print(cp.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  87.7%
#     Precision:     70.6%
#     Recall:        67.8%
#     F-Measure:     69.2%

ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     70.6%%
    Recall:        67.8%%
    F-Measure:     69.2%%


In [15]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [16]:
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])

In [17]:
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])

In [18]:
unigram_chunker = UnigramChunker(train_sents)

In [19]:
print(unigram_chunker.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  92.9%
#     Precision:     79.9%
#     Recall:        86.8%
#     F-Measure:     83.2%

ChunkParse score:
    IOB Accuracy:  92.9%%
    Precision:     79.9%%
    Recall:        86.8%%
    F-Measure:     83.2%%


In [20]:
postags = sorted(set(pos for sent in train_sents
                     for (word,pos) in sent.leaves()))

In [21]:
print(unigram_chunker.tagger.tag(postags))
# [('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'),
#  (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'),
#  ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'),
#  ('JJ', 'I-NP'), ('JJR', 'B-NP'), ('JJS', 'I-NP'), ('MD', 'O'),
#  ('NN', 'I-NP'), ('NNP', 'I-NP'), ('NNPS', 'I-NP'), ('NNS', 'I-NP'),
#  ('PDT', 'B-NP'), ('POS', 'B-NP'), ('PRP', 'B-NP'), ('PRP$', 'B-NP'),
#  ('RB', 'O'), ('RBR', 'O'), ('RBS', 'B-NP'), ('RP', 'O'), ('SYM', 'O'),
#  ('TO', 'O'), ('UH', 'O'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'),
#  ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'B-NP'),
#  ('WP', 'B-NP'), ('WP$', 'B-NP'), ('WRB', 'O'), ('``', 'O')]

[('#', 'B-NP'), ('$', 'B-NP'), ("''", 'O'), ('(', 'O'), (')', 'O'), (',', 'O'), ('.', 'O'), (':', 'O'), ('CC', 'O'), ('CD', 'I-NP'), ('DT', 'B-NP'), ('EX', 'B-NP'), ('FW', 'I-NP'), ('IN', 'O'), ('JJ', 'I-NP'), ('JJR', 'B-NP'), ('JJS', 'I-NP'), ('MD', 'O'), ('NN', 'I-NP'), ('NNP', 'I-NP'), ('NNPS', 'I-NP'), ('NNS', 'I-NP'), ('PDT', 'B-NP'), ('POS', 'B-NP'), ('PRP', 'B-NP'), ('PRP$', 'B-NP'), ('RB', 'O'), ('RBR', 'O'), ('RBS', 'B-NP'), ('RP', 'O'), ('SYM', 'O'), ('TO', 'O'), ('UH', 'O'), ('VB', 'O'), ('VBD', 'O'), ('VBG', 'O'), ('VBN', 'O'), ('VBP', 'O'), ('VBZ', 'O'), ('WDT', 'B-NP'), ('WP', 'B-NP'), ('WP$', 'B-NP'), ('WRB', 'O'), ('``', 'O')]


In [25]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [27]:
bigram_chunker = BigramChunker(train_sents)

In [28]:
print(bigram_chunker.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  93.3%
#     Precision:     82.3%
#     Recall:        86.8%
#    F-Measure:     84.5%

ChunkParse score:
    IOB Accuracy:  93.3%%
    Precision:     82.3%%
    Recall:        86.8%%
    F-Measure:     84.5%%


In [29]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.MaxentClassifier.train(
            train_set, algorithm='megam', trace=0)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [30]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}

In [31]:
chunker = ConsecutiveNPChunker(train_sents)

LookupError: 

===========================================================================
NLTK was unable to find the megam file!
Use software specific configuration paramaters or set the MEGAM environment variable.

  For more information on megam, see:
    <http://www.umiacs.umd.edu/~hal/megam/index.html>
===========================================================================

In [32]:
print(chunker.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  92.9%
#     Precision:     79.9%
#     Recall:        86.7%
#     F-Measure:     83.2%

NameError: name 'chunker' is not defined

In [33]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "prevpos": prevpos}

In [34]:
chunker = ConsecutiveNPChunker(train_sents)

LookupError: 

===========================================================================
NLTK was unable to find the megam file!
Use software specific configuration paramaters or set the MEGAM environment variable.

  For more information on megam, see:
    <http://www.umiacs.umd.edu/~hal/megam/index.html>
===========================================================================

In [35]:
print(chunker.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  93.6%
#     Precision:     81.9%
#     Recall:        87.2%
#     F-Measure:     84.5%

NameError: name 'chunker' is not defined

In [36]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "word": word, "prevpos": prevpos}

In [37]:
chunker = ConsecutiveNPChunker(train_sents)

LookupError: 

===========================================================================
NLTK was unable to find the megam file!
Use software specific configuration paramaters or set the MEGAM environment variable.

  For more information on megam, see:
    <http://www.umiacs.umd.edu/~hal/megam/index.html>
===========================================================================

In [38]:
print(chunker.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  94.5%
#     Precision:     84.2%
#     Recall:        89.4%

NameError: name 'chunker' is not defined

In [39]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    if i == len(sentence)-1:
        nextword, nextpos = "<END>", "<END>"
    else:
        nextword, nextpos = sentence[i+1]
    return {"pos": pos,
            "word": word,
            "prevpos": prevpos,
            "nextpos": nextpos,
            "prevpos+pos": "%s+%s" % (prevpos, pos),
            "pos+nextpos": "%s+%s" % (pos, nextpos),
            "tags-since-dt": tags_since_dt(sentence, i)}

In [40]:
def tags_since_dt(sentence, i):
    tags = set()
    for word, pos in sentence[:i]:
        if pos == 'DT':
            tags = set()
        else:
            tags.add(pos)
    return '+'.join(sorted(tags))

In [41]:
chunker = ConsecutiveNPChunker(train_sents)

LookupError: 

===========================================================================
NLTK was unable to find the megam file!
Use software specific configuration paramaters or set the MEGAM environment variable.

  For more information on megam, see:
    <http://www.umiacs.umd.edu/~hal/megam/index.html>
===========================================================================

In [42]:
print(chunker.evaluate(test_sents))
# ChunkParse score:
#     IOB Accuracy:  96.0%
#     Precision:     88.6%
#     Recall:        91.0%
#     F-Measure:     89.8%

NameError: name 'chunker' is not defined